In [10]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import numpy as np

### Conexión a Link_de_Reasignacion

In [2]:
try: 
    conexion = mysql.connector.connect(
        host="172.18.50.9",
        port=3306,
        user="consultapedidos",
        password="#Cr9zV3rd3*",
        db="domicilios"
    
    )
    
    if conexion.is_connected(): 
        print("Conexión exitosa")
        infoServer = conexion.get_server_info()
        print("Info del servidor: ", infoServer)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT database();")
        registro = cursor.fetchone()
        print("Conectado a la BD: ",registro)
        
        
except Error as ex:
    print("Error durante conexión:", ex)

    

Conexión exitosa
Info del servidor:  5.7.23-log
Conectado a la BD:  ('domicilios',)


In [3]:
%%time
periodo = 202209

cur = conexion.cursor()
cur.execute("SELECT * FROM domi_pedidos WHERE periodo in ('202105')")

data =cur.fetchall()
df = pd.DataFrame(data)
df.columns = cur.column_names

CPU times: total: 3.98 s
Wall time: 27.3 s


### VENTASMARGEN

In [2]:
try: 
    conexion = mysql.connector.connect(
        host="172.18.50.54",
        port=3306,
        user="operaciones",
        password="operaciones3",
        db="ventas"
    
    )
    
    if conexion.is_connected(): 
        print("Conexión exitosa")
        infoServer = conexion.get_server_info()
        print("Info del servidor: ", infoServer)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT database();")
        registro = cursor.fetchone()
        print("Conectado a la BD: ",registro)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT  Mes ,Dia, NumeroTicket, Sucursal,Venta, Clasifica1,identifUsuario, periodo FROM ventasdiario_c WHERE periodo in ('202301') and Clasifica1 in ('Droguería','Droguerías','Drogueria','Droguerias','droguería','droguerías','drogueria','droguerias');")
        data = cursor.fetchall()
        df_drog=pd.DataFrame(data)
        df_drog.columns= cursor.column_names
        
except Error as ex:
    print("Error durante conexión:", ex)


Conexión exitosa
Info del servidor:  5.7.21-log
Conectado a la BD:  ('ventas',)


In [11]:
try: 
    conexion = mysql.connector.connect(
        host="172.18.50.54",
        port=3306,
        user="operaciones",
        password="operaciones3",
        db="ventas"
    
    )
 
    if conexion.is_connected(): 
        print("Conexión exitosa")
        infoServer = conexion.get_server_info()
        print("Info del servidor: ", infoServer)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT database();")
        registro = cursor.fetchone()
        print("Conectado a la BD: ",registro)
        
        cursor.execute("SELECT Canal,Sucursal, Dia, NumeroTicket, Venta, Clasifica1, periodo, identifUsuario FROM ventasdiario_c WHERE periodo in ('202302') and Clasifica1 in ('Ecommerce','Mostrador','Call Center');")
        data = cursor.fetchall()
        df_mostrador=pd.DataFrame(data)
        df_mostrador.columns= cursor.column_names
        df_mostrador
        
except Error as ex:
    print("Error durante conexión:", ex)

Conexión exitosa
Info del servidor:  5.7.21-log
Conectado a la BD:  ('ventas',)


In [13]:
df_mostrador_unico = df_mostrador.drop_duplicates(subset = ["NumeroTicket"])
df_mostrador_unico = df_mostrador_unico[df_mostrador_unico["Venta"]>0]

In [14]:
pd.pivot_table(df_mostrador_unico, index = ["Clasifica1"],values = ["NumeroTicket"],aggfunc = "count")

,NumeroTicket
Clasifica1,
Call Center,49571
Ecommerce,39913
Mostrador,23342


In [15]:
pd.pivot_table(df_mostrador, index = ["Clasifica1"],values = ["Venta"],aggfunc = np.sum)

,Venta
Clasifica1,
Call Center,4.900511e+09
Ecommerce,3.869077e+09
Mostrador,2.982074e+09


In [7]:
resumen_venta = pd.pivot_table(df_mostrador,index = ["Sucursal"],columns = ["Clasifica1"],values = ["Venta"],aggfunc = np.sum)
resumen_venta.to_excel(r"C:\Users\luis.montoya\Downloads\Venta_Sucursal2022.xlsx")

In [4]:
try: 
    conexion = mysql.connector.connect(
        host="172.18.50.54",
        port=3306,
        user="operaciones",
        password="operaciones3",
        db="ventas"
    
    )
    
    if conexion.is_connected(): 
        print("Conexión exitosa")
        infoServer = conexion.get_server_info()
        print("Info del servidor: ", infoServer)
        
        cursor=conexion.cursor()
        cursor.execute("SELECT database();")
        registro = cursor.fetchone()
        print("Conectado a la BD: ",registro)
        
        cursor.execute("SELECT  Sucursal, identifUsuario, NumeroTicket, CodArticulo, Venta, Ano, Mes, Dia, clasifica, periodo FROM ventasdiariogeoposi WHERE periodo in ('202212') and CodArticulo in (158135,158136);")
        data = cursor.fetchall()
        df=pd.DataFrame(data)
        df.columns= cursor.column_names
        df
        
except Error as ex:
    print("Error durante conexión:", ex)

Conexión exitosa
Info del servidor:  5.7.21-log
Conectado a la BD:  ('ventas',)


In [5]:
df_MDomicilios = df[df["clasifica"] == "Mostrador"]
df_MDrogueria = df[df["clasifica"] == "Drogueria"]

In [6]:
marcacion_MDrogueria = pd.merge(df_drog,df_MDrogueria,left_on ="NumeroTicket",right_on="NumeroTicket",how="left")
marcacion_MDrogueria = marcacion_MDrogueria[~marcacion_MDrogueria.CodArticulo.isnull()]

In [7]:
marcacion_MDomicilios = pd.merge(df_mostrador,df_MDomicilios,left_on ="NumeroTicket",right_on="NumeroTicket",how="left")
marcacion_MDomicilios = marcacion_MDomicilios[~marcacion_MDomicilios.CodArticulo.isnull()]

In [8]:
marcacion_MDomicilios.to_excel(r"C:\Users\luis.montoya\Downloads\MostradorDomicilios.xlsx")
marcacion_MDrogueria.to_excel(r"C:\Users\luis.montoya\Downloads\MostradorDrogueria.xlsx")

In [ ]:
############################################################################################################################